In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# 基础路径
base_path = Path(r"C:\Users\田\Desktop\python实操\kaggle\Natural Language Processing with Disaster Tweets")  # 你的原始路径

In [2]:
sample_submission = pd.read_csv(base_path / 'sample_submission.csv')
test = pd.read_csv(base_path / 'test.csv')
train = pd.read_csv(base_path / 'train.csv')

> **环境**: Python 3.13, PyTorch 2.x, Transformers 4.x

<big>sft<big>

In [3]:
%%time
import pandas as pd
import numpy as np
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# ==============================
# 配置参数
# ==============================
CONFIG = {
    'seed': 42,
    'max_len': 128,
    'batch_size': 32,
    'epochs': 2,
    'learning_rate': 2e-5,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'model_path': r'C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased'
}

# ==============================
# 文本预处理
# ==============================
def clean_text(text: str) -> str:
    """清理推文文本：移除URL、@提及、#标签，并规范化空格"""
    text = str(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)      # 移除链接
    text = re.sub(r'@\w+', '', text)                       # 移除@用户
    text = re.sub(r'#', '', text)                          # 移除井号（保留词）
    text = re.sub(r'\s+', ' ', text).strip()               # 合并多余空格
    return text

def build_prompt(text: str) -> str:
    """构建分类任务提示"""
    return f"判断以下推文是否描述真实灾难事件：{text}"

# ==============================
# 自定义数据集
# ==============================
class DisasterTweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        prompt = build_prompt(self.texts[idx])
        encoding = self.tokenizer(
            prompt,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        else:
            item['labels'] = torch.tensor(0, dtype=torch.long)  # 测试集占位

        return item

# ==============================
# 训练与评估
# ==============================
def train_one_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='macro')
    return f1, all_preds

# ==============================
# 主流程
# ==============================
def main(train_df, test_df, sample_sub):
    print("🚀 启动灾难推文分类训练流程")
    print(f"设备: {CONFIG['device']}")
    
    # 数据清洗
    train_df['clean_text'] = train_df['text'].apply(clean_text)
    test_df['clean_text'] = test_df['text'].apply(clean_text)

    # 划分训练/验证集
    X_train, X_val, y_train, y_val = train_test_split(
        train_df['clean_text'].values,
        train_df['target'].values,
        test_size=0.2,
        random_state=CONFIG['seed'],
        stratify=train_df['target']
    )

    # 初始化 tokenizer 和模型
    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_path'])
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model = AutoModelForSequenceClassification.from_pretrained(
        CONFIG['model_path'], num_labels=2
    )
    model.resize_token_embeddings(len(tokenizer))
    model.to(CONFIG['device'])

    # 构建数据加载器
    train_dataset = DisasterTweetDataset(X_train, y_train, tokenizer, CONFIG['max_len'])
    val_dataset = DisasterTweetDataset(X_val, y_val, tokenizer, CONFIG['max_len'])
    test_dataset = DisasterTweetDataset(test_df['clean_text'].values, None, tokenizer, CONFIG['max_len'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

    # 优化器与学习率调度
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'])
    total_steps = len(train_loader) * CONFIG['epochs']
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_steps * 0.1),
        num_training_steps=total_steps
    )

    # 训练循环
    best_f1 = 0.0
    for epoch in range(CONFIG['epochs']):
        print(f"\n{'='*40}\nEpoch {epoch + 1}/{CONFIG['epochs']}\n{'='*40}")

        train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, CONFIG['device'])
        val_f1, _ = evaluate(model, val_loader, CONFIG['device'])

        print(f"训练损失: {train_loss:.4f} | 验证F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), f"best_model_epoch{epoch+1}.pth")
            print(f"✅ 保存最优模型 (F1={best_f1:.4f})")

    # 测试集预测
    print("\n🔮 生成测试集预测...")
    model.load_state_dict(torch.load(f"best_model_epoch{CONFIG['epochs']}.pth"))
    model.eval()

    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(CONFIG['device'])
            attention_mask = batch['attention_mask'].to(CONFIG['device'])
            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())

    # 保存提交文件
    submission = sample_sub.copy()
    submission['target'] = predictions[:len(submission)]
    submission.to_csv('submission.csv', index=False)
    print(f"✅ 提交文件已保存，最佳验证F1: {best_f1:.4f}")

    return submission

# ==============================
# 入口点（假设 train/test/sample_submission 已定义）
# ==============================
if __name__ == "__main__":
    try:
        # 注意：此处需确保 train, test, sample_submission 已在全局作用域中定义
        # （例如从 Kaggle 环境或本地 CSV 加载）
        submission = main(train, test, sample_submission)

        # 输出预测统计
        print(f"\n📊 预测分布: 非灾难={sum(submission['target']==0)}, 灾难={sum(submission['target']==1)}")

    except Exception as e:
        print(f"❌ 训练失败: {str(e)}")
        import traceback
        traceback.print_exc()

🚀 启动灾难推文分类训练流程
设备: cuda


W0130 22:21:34.084000 4512 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/2
训练损失: 0.4858 | 验证F1: 0.8371
✅ 保存最优模型 (F1=0.8371)

Epoch 2/2
训练损失: 0.3538 | 验证F1: 0.8496
✅ 保存最优模型 (F1=0.8496)

🔮 生成测试集预测...
✅ 提交文件已保存，最佳验证F1: 0.8496

📊 预测分布: 非灾难=2016, 灾难=1247
CPU times: total: 3min 49s
Wall time: 4min 28s
